In [ ]:
!pip install finufft

In [ ]:
import numpy as np
from astropy.table import Table
import scipy.sparse.linalg as sp
import finufft
import pylab as plt

In [ ]:
IMAX = 40 # maximum integer wave number
DELTAK = 2. * np.pi / 10000.0 # wave number spacing in inverse pixels

In [ ]:
data = Table.read("dxyPixels.csv", format='ascii.csv')

In [ ]:
plt.plot(data["x"], data["y"], "k.")

In [ ]:
# unpack the data
xs = data["x"]
ys = data["y"]
dxs = data["dx"]
dys = data["dy"]
n = len(xs)

In [ ]:
# rescale position inputs so they are in the range -pi, pi.
shiftx = np.mean(xs)
scalexy = np.pi / 2600.
sxs = (xs - shiftx) * scalexy
shifty = np.mean(ys)
sys = (ys - shifty) * scalexy
print(np.min(sxs), np.max(sxs), np.min(sys), np.max(sys))

In [ ]:
# Make least-squares (pseudo-inverse) code for finufft2d2

def nufft2d2_pinv(x, y, c, Ns):
    """
    The pseudo-inverse of `nufft2d2()`.
  
    ## Inputs:
    - `x, y`  positions in 2D (ought to be in [-pi, pi] or so)
    - `c`     data
    - `Ns`    2-tuple `(Nx, Ny)`
  
    ## Outputs:
    - `fhat`  best-fit Fourier coefficients
  
    ## Notes:
    - This finds the minimum-least-squares amplitudes f to explain data c.
    
    ## Bugs:
    - Completely untested
    """
    M, N = len(x), Ns[0] * Ns[1]
    f0 = finufft.nufft2d1(x, y, c, Ns, eps=FEPS).flatten()
    R = lambda f: finufft.nufft2d2(x, y, f.reshape(Ns), eps=FEPS)
    RT = lambda c: finufft.nufft2d1(x, y, c, Ns, eps=FEPS).flatten()
    RR = sp.LinearOperator((M, N), matvec=R, rmatvec=RT, dtype=complex)
    res = sp.lsqr(RR, c, x0=f0, atol=ATOL, btol=BTOL)
    print("nufft2d2_pinv: completed in", res[2], "iterations")
    return res[0].reshape(Ns)

FEPS = 1e-8
ATOL = 1e-8
BTOL = 1e-8

In [ ]:
# make train-test split
np.random.seed(42)
rands = np.random.uniform(size=n)
train = rands <= 0.8
test = rands > 0.8
print(np.sum(train), np.sum(test))

In [ ]:
# do the least-squares fit for the dxs in the training set
Nx, Ny = 80, 80
fhat_dxs = nufft2d2_pinv(sxs[train], sys[train], dxs[train].astype(complex), (Nx, Ny))

In [ ]:
# Now predict the dxs in test set
dxs_hat = finufft.nufft2d2(sxs[test], sys[test], fhat_dxs).real

In [ ]:
# Now compare!
print("original dx (test set) RMS:", np.sqrt(np.mean(dxs[test] ** 2)))
print("dx - dx_hat (test set) RMS:", np.sqrt(np.mean((dxs[test] - dxs_hat) ** 2)))
print("dx - dx_hat (test set) MAD:", np.sqrt(np.median((dxs[test] - dxs_hat) ** 2)))

In [ ]:
# do the least-squares fit for the dys in the training set
fhat_dys = nufft2d2_pinv(sxs[train], sys[train], dys[train].astype(complex), (Nx, Ny))

In [ ]:
# Now predict the dxs in test set
dys_hat = finufft.nufft2d2(sxs[test], sys[test], fhat_dys).real

In [ ]:
# Now compare!
print("original dy (test set) RMS:", np.sqrt(np.mean(dys[test] ** 2)))
print("dy - dy_hat (test set) RMS:", np.sqrt(np.mean((dys[test] - dys_hat) ** 2)))
print("dy - dy_hat (test set) MAD:", np.sqrt(np.median((dys[test] - dys_hat) ** 2)))